In [ ]:
import pandas as pd
import fiona
import geopandas as gpd
from shapely.geometry import LineString, Point
import osmnx  as ox
import networkx as nx
import numpy as np
import folium
import matplotlib.pyplot as plt
ox.config(use_cache=True, log_console=True)
import numpy as np
import random as rand

In [ ]:
#Load local road map of Dornbirn
filepath = "./data/dornbirn_hemi.graphml"
G = ox.load_graphml(filepath)
# Get the edges/network as GeoDataFrame
edges = ox.graph_to_gdfs(G, nodes=False)
edges.crs

In [ ]:
# # Alternatively download new map from OpenStreetMap
#network = 'drive'
#G = ox.graph_from_point((47.40518,9.74442),dist=5000,simplify=True, network_type= network, clean_periphery=True)
## save graph
#filepath = "./data/dornbirn_hemi.graphml"
#ox.save_graphml(G, filepath)

In [ ]:
# Plot road map
G=G.to_undirected()
ox.plot_graph(G, bgcolor='w')

In [ ]:
## Check coordinate system of graph as Geodataframe
Ggdf=ox.graph_to_gdfs(G)
# 
# pG = ox.project_graph(G,to_crs='epsg:4326')
# pGgdf=ox.graph_to_gdfs(pG)
Ggdf[0].crs

## Build data frame of locations in Dornbirn
- including the pairwise shortest distances
- including the pairwise shortest path connections

In [ ]:
# Build data frame of locations in Dornbirn
name=['FH Vorarlberg','Messepark','Bahnhof','Kapelle Mühlebach','Spielboden','Oberfallenberg','Olympiazentrum','VS Wallenmahd','H. Gmeiner Park', 'Friedhof Rohrbach']
latitude=[47.40518,47.41080,47.41692,47.39615,47.42151,47.42219,47.42010,47.39259,47.40391,47.42497]
longitude=[9.74442,9.71260,9.73912,9.74429,9.73575, 9.76998, 9.72072,9.73299,9.72576,9.72651]

df_stops = pd.DataFrame(
    {'Name' : name,
     'latitude' : latitude,
     'longitude' : longitude
    })
df_stops.head()

# number of locations
n = len(name)
print(n,df_stops)

In [ ]:
## Compute closest graph nodes in G associated with the locations in the dataframe above
# coordinates to list
locationlist = df_stops[['latitude', 'longitude']].values.tolist()
locationlist
# convert coordinates to Geoseries Point objects in Graph related coordinate system
pointlist=[Point(l) for l in locationlist]
pts = gpd.GeoSeries(pointlist, crs='epsg:4326')
# identify nearest nodes to Point objects in road map graph G
nearest_nodes = [ox.distance.nearest_nodes(G, pt.y, pt.x, return_dist=0) for pt in pts]
nearest_nodes

In [ ]:
# Compute pairwise distances and corresponding paths between all locations with respect to the graph G and add them to the dataframe
Dist=list()
Path=list()
for i in range(0,n):
    orig=nearest_nodes[i]
    leng=list()
    path=list()
    for j in range(0,n):
        dest=nearest_nodes[j]
        if i==j:
            leng.append(0)
            path.append(nx.subgraph(G,ox.distance.shortest_path(G,orig,dest)))
        else:
            leng.append(nx.shortest_path_length(G, source=orig, target=dest, weight='length'))
            path.append(nx.subgraph(G,ox.distance.shortest_path(G,orig,dest)))
    Dist.append(leng)
    Path.append(path)
# Convert into symmetrical matrix and add to dataframe
D=np.eye(n)
for i in range(0,n):
    for j in range(i,n):
        D[i,j]=Dist[i][j]
Dt=D.transpose()
D=D+Dt
# round distances
D=np.round(D,0).tolist()

# use symmetric path representations
for j in range(0,n):
    for i in range(0,j+1):
        Path[i][j]=Path[j][i]

# write to dataframe
for k in range(0,n):
    df_stops['dist_to_loc' + str(k)]=D[k]
for k in range(0,n):
    df_stops['path_to_loc' + str(k)]=Path[k]


In [ ]:
df_stops.head()

## Tour creation and visualization

In [ ]:
# determine the subgraph of G for a given path (based on the distances in the dataframe)
def gatherSubgraph(tour,df,G):
    n=len(tour)
    tour.append(0)
    subgraph=list()
    for i in range(0,n):
        subpath=df.iloc[tour[i],13+tour[i+1]]
        subgraph = subgraph + list(subpath.nodes)
    sG = nx.subgraph(G,subgraph)
    return sG

In [ ]:
# Demo: subgraph in G corresponding to path and df
tour = [0,3,7,6,1,8,9,4,2,5,1] #ner optimal: [0, 2, 5, 4, 9, 6, 1, 8, 7, 3]
sG=gatherSubgraph(tour,df_stops,G)
sG

In [ ]:
# printing routine for graph and subgraph (tour)
def PrintRoute(sG,df_stops,G):
    locationlist = df_stops[['latitude', 'longitude']].values.tolist()
    map0=folium.Map(location=locationlist[0], zoom_start=11)
    mapx=ox.folium.plot_graph_folium(G, graph_map=map0, popup_attribute=None, tiles='cartodbpositron', fit_bounds=True, color='blue', weight=1, opacity=0.6)
    map=ox.folium.plot_graph_folium(sG, graph_map=mapx, popup_attribute=None, tiles='cartodbpositron', fit_bounds=True,  color='red', weight=2, opacity=1)
    for point in range(0, len(locationlist)):
        if point == 0:
            folium.Marker(locationlist[point], popup=df_stops['Name'][point],icon=folium.Icon(color="red", icon="info-sign")).add_to(map)
        else:
            folium.Marker(locationlist[point], popup=df_stops['Name'][point],icon=folium.Icon(color="green")).add_to(map)
    map.fit_bounds(map.get_bounds(), padding=(10, 10))
    return map

In [ ]:
PrintRoute(sG,df_stops,G)

## GA for TSP
The follwoing cells contain the subroutines of the Genetic Algorithm (GA) used to determine an (near) optimal tour.

In [ ]:
# Objective function measureing the tour length in meters [m]
def objFun(tour,df):
    tourn=tour.copy()
    tourn.append(0)
    dist=0
    for i in range(0,len(tourn)-1):
        dist = dist + df.iloc[tourn[i],3+tourn[i+1]]
    return dist

In [ ]:
print(tour)
tour_len=objFun(tour,df_stops)
print(tour_len)

In [ ]:
# mutation operator used within the GA to randomly vary two consecutive stops of a tour
def mutate(tour):
    ntour=tour.copy()
    idx = np.random.randint(1, len(tour)-1)
    ntour[idx], ntour[idx+1] = ntour[idx+1], ntour[idx]
    return ntour

# demo:
tour1 = [0,4,5,1,2,3,6,7,8,9]
tour2 = [0,3,4,2,1,5,7,6,9,8]
print(tour1,tour2)

mutated_tour=mutate(tour1)
print(mutated_tour)

In [ ]:
# (ordered) crossover operator to recombine two potential tours within the GA
def crx_tours(tourA,tourB):
    n=len(tourB)
    new_tourA= [0] * n
    new_tourB= [0] * n
    # determine center of cut-points
    idx = np.random.randint(2,n-1)
    keep = [0,idx-1,idx,idx+1]
    nlist=list(range(n))
    change_p = [val for val in nlist if val not in keep and val > idx]
    change_n = [val for val in nlist if val not in keep and val < idx]
    for i in keep:
        new_tourA[i]=tourA[i]
        new_tourB[i]=tourB[i]
    for i in change_p:
        vals1 = [val for val in tourB if val not in new_tourA]
        new_tourA[i] = vals1[0]
        vals2 = [val for val in tourA if val not in new_tourB]
        new_tourB[i] = vals2[0]
    for i in change_n:
        vals1 = [val for val in tourB if val not in new_tourA]
        new_tourA[i] = vals1[0]
        vals2 = [val for val in tourA if val not in new_tourB]
        new_tourB[i] = vals2[0]
    return new_tourA,new_tourB

#demo:
print(tour1,tour2)
crossed_tour1,crossed_tour2 = crx_tours(tour1, tour2)
print(crossed_tour1,crossed_tour2)

In [ ]:
initial_tour =[0,1,2,3,4,5,6,7,8,9]
initial_tour_len = objFun(initial_tour,df_stops)
# GA for TSP implementation
def ga4tsp(df,pop_size,iter_n):
    parent=initial_tour
    n=len(parent)
    best = objFun(parent,df)
    par=parent.copy()
    for j in range(0,iter_n):
        Offsp=list()
        Eval=list()
        for i in range(0,pop_size):
            test=mutate(par)
            Offsp.append(test.copy())
            Eval.append(objFun(Offsp[i],df))
        idx = np.argsort(Eval)
        par1=Offsp[idx[0]]
        par2=Offsp[idx[1]]
        val1=Eval[idx[0]]
        val2=Eval[idx[1]]
        parA,parB = crx_tours(par1,par2)
        val3 =objFun(parA,df)
        val4 =objFun(parB,df)
        listv=[val1,val2,val3,val4]
        pop = [par1,par2,parA,parB]
        ev=np.argmin(listv)
        if listv[ev] <= best:
            best = listv[ev]
            bestp = pop[ev]
        par=pop[ev]    
        # print(par,best)            
    return bestp, best

In [ ]:
# run GA
final_tour, final_tour_len=ga4tsp(df_stops,10,500)

In [ ]:
print(initial_tour,initial_tour_len)
print(final_tour,final_tour_len)

### Comparison of initial and final tour

In [ ]:
# visualization of initial tour
initialG = gatherSubgraph(initial_tour,df_stops,G)
PrintRoute(initialG,df_stops,G)

In [ ]:
# visualization of final tour
finalG = gatherSubgraph(final_tour,df_stops,G)
PrintRoute(finalG,df_stops,G)